In [3]:
from numpy import * 

# 数据导入
def loadDataSet(fileName):
    dataMat = []
    fr = open(fileName)
    for line in fr.readlines():
        curLine = line.strip().split('\t')
        fltLine = list(map(float, curLine))
        dataMat.append(fltLine)
    return mat(dataMat)

# 计算距离
def distEclud(vecA, vecB):
    return sqrt(sum(power(vecA-vecB, 2)))

# 随机质心
def randCent(dataSet, k):
    n = shape(dataSet)[1]
    centroids = mat(zeros((k, n)))
    for j in range(n):
        minJ = min(dataSet[:, j])
        rangeJ = float(max(dataSet[:, j]) - minJ)
        centroids[:, j] =  minJ + rangeJ * random.rand(k,1)
    return centroids


dataMat = loadDataSet('testSet.txt')
randCent(dataMat, 2)

matrix([[-4.52320693, -0.62144201],
        [-5.01031465,  2.7449943 ]])

In [4]:
"""
    k均值据类算法
    算法会创建k个质心，然后将每个点分配到最近的质心，在重新计算质心。该过程重复执行，知道质心不再改变
"""
def KMeans(dataSet, k, distMeas=distEclud, createCent=randCent):
    m = shape(dataSet)[0] 
    clusterAssment = mat(zeros((m, 2)))
    centroids = createCent(dataSet, k)
    clusterChanged = True
    while clusterChanged:
        clusterChanged = False
        for i in range(m):
            minDist = inf; minIndex = -1
            for j in range(k):
                distJI = distMeas(centroids[j, :], dataSet[i, :])
                if distJI < minDist:
                    minDist = distJI; minIndex = j
            if clusterAssment[i, 0] != minIndex:
                clusterChanged = True
            clusterAssment[i, :] = minIndex, minDist**2
#         print(clusterAssment)
        for cent in range(k):
            ptsInClust = dataSet[nonzero(clusterAssment[:, 0].A==cent)[0]]
            centroids[cent, :] = mean(ptsInClust, axis=0)
    return centroids, clusterAssment


centroids, clusterAssment = KMeans(dataMat, 4)
centroids

matrix([[ 2.6265299 ,  3.10868015],
        [-3.53973889, -2.89384326],
        [-2.46154315,  2.78737555],
        [ 2.65077367, -2.79019029]])

## 避免聚类收敛到局部最小
* 合并最近的质心
* 合并两个使得SSE增幅最小的质心

In [14]:
mean(dataMat, axis=0).tolist()[0]

[-0.10361321250000004, 0.05430119999999998]

In [25]:
"""
    二分K 均值算法
    将所有点作为一个簇，然后将该簇一分为二。之后选择其中一个簇（可以最大程度降低SSE（Sum of Square Error:误差平方和））
    ，上述基于SSE不断重复，直到得到指定数目的簇
"""
def biKmeans(dataSet, k, distMeans=distEclud):
    m = shape(dataSet)[0]
    clusterAssment = mat(zeros((m,2)))
    centroid0 = mean(dataSet, axis=0).tolist()[0]
#     print(centroid0)
    centList = [centroid0]
#     print(centList)
    for j in range(m):
        clusterAssment[j, 1] = distMeans(mat(centroid0), dataSet[j, :]) ** 2
    while (len(centList) < k):
        lowestSSE = inf
        for i in range(len(centList)):
            ptsInCurrCluster = dataSet[nonzero(clusterAssment[:,0].A==i)[0], :]
            centroidMat, splitClustAss = KMeans(ptsInCurrCluster, 2, distMeans)
            sseSplit = sum(splitClustAss[:,1])
            sseNotSplit = sum(clusterAssment[nonzero(clusterAssment[:,0].A!=i)[0], 1])
            print("sseSplit and notSplit", sseSplit, sseNotSplit)
            if (sseSplit + sseNotSplit) < lowestSSE:
                bestCentToSplit = i
                bestNewCents = centroidMat
                bestClustAss = splitClustAss.copy()
                lowestSSE = sseSplit + sseNotSplit
        bestClustAss[nonzero(bestClustAss[:, 0].A==1)[0], 0] = len(centList)
        bestClustAss[nonzero(bestClustAss[:, 0].A==0)[0], 0] = bestCentToSplit
        print("The bestCentToSplit is :", bestCentToSplit)
        print("The len os bestClustAss is :", len(bestClustAss))
        centList[bestCentToSplit] = bestNewCents[0,:]
        centList.append(bestNewCents[1, :])
        clusterAssment[nonzero(clusterAssment[:,0].A==bestCentToSplit)[0],:] = bestClustAss
    return centList, clusterAssment
        
            

biKmeans(dataMat, 4)

sseSplit and notSplit 792.9168565373268 0.0
The bestCentToSplit is : 0
The len os bestClustAss is : 80
sseSplit and notSplit 66.36683512000786 466.63278133614426
sseSplit and notSplit 83.5874695564185 326.2840752011824
The bestCentToSplit is : 1
The len os bestClustAss is : 40
sseSplit and notSplit 66.36683512000786 83.5874695564185
sseSplit and notSplit 19.39517908946145 377.2703018926498
sseSplit and notSplit 31.62960698022571 358.8853180661336
The bestCentToSplit is : 0
The len os bestClustAss is : 40


([matrix([[2.6265299 , 3.10868015]]),
  matrix([[-3.38237045, -2.9473363 ]]),
  matrix([[ 2.80293085, -2.7315146 ]]),
  matrix([[-2.46154315,  2.78737555]])],
 matrix([[0.00000000e+00, 2.32019150e+00],
         [3.00000000e+00, 1.39004893e+00],
         [2.00000000e+00, 6.63839104e+00],
         [1.00000000e+00, 4.16140951e+00],
         [0.00000000e+00, 2.76967820e+00],
         [3.00000000e+00, 2.80101213e+00],
         [2.00000000e+00, 5.85909807e+00],
         [1.00000000e+00, 1.50646425e+00],
         [0.00000000e+00, 2.29348924e+00],
         [3.00000000e+00, 6.45967483e-01],
         [2.00000000e+00, 1.74010499e+00],
         [1.00000000e+00, 3.77769471e-01],
         [0.00000000e+00, 2.51695402e+00],
         [3.00000000e+00, 1.38716420e-01],
         [2.00000000e+00, 9.47633071e+00],
         [1.00000000e+00, 9.97310599e+00],
         [0.00000000e+00, 2.39726914e+00],
         [3.00000000e+00, 3.10242360e+00],
         [2.00000000e+00, 4.11084375e-01],
         [1.00000000e+00